In [1]:
import pandas as pd

df_pass = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-bioasq/data/passages.parquet/part.0.parquet")
df_test = pd.read_parquet("hf://datasets/rag-datasets/rag-mini-bioasq/data/test.parquet/part.0.parquet")

In [2]:
print(df_pass.columns)
print(len(df_pass))
df_pass.head()

Index(['passage'], dtype='object')
40221


,passage
id,
9797,New data on viruses isolated from patients wit...
11906,We describe an improved method for detecting d...
16083,We have studied the effects of curare on respo...
23188,Kinetic and electrophoretic properties of 230-...
23469,Male Wistar specific-pathogen-free rats aged 2...


In [3]:
from langchain_core.documents import Document
print(df_pass.iloc[0])
docs = [Document(page_content=row["passage"]) for _, row in df_pass.iterrows()]

passage    New data on viruses isolated from patients wit...
Name: 9797, dtype: string


In [4]:
df_test.head()

,question,answer,relevant_passage_ids
id,,,
0,Is Hirschsprung disease a mendelian or a multi...,"Coding sequence mutations in RET, GDNF, EDNRB,...","[20598273, 6650562, 15829955, 15617541, 230011..."
1,List signaling molecules (ligands) that intera...,The 7 known EGFR ligands are: epidermal growt...,"[23821377, 24323361, 23382875, 22247333, 23787..."
2,Is the protein Papilin secreted?,"Yes, papilin is a secreted protein","[21784067, 19297413, 15094122, 7515725, 332004..."
3,Are long non coding RNAs spliced?,Long non coding RNAs appear to be spliced thro...,"[22955974, 21622663, 22707570, 22955988, 24285..."
4,Is RANKL secreted from the cells?,Receptor activator of nuclear factor κB ligand...,"[22867712, 23827649, 21618594, 23835909, 24265..."


In [11]:
ids_list = df_test["relevant_passage_ids"]
print(ids_list[0])
max = 0
max_idx = 0

for idx, i in enumerate(ids_list):
    i_list = eval(i)
    if len(i_list) > max:
        max = len(i_list)
        max_idx = i_list
max, max_idx


[20598273, 6650562, 15829955, 15617541, 23001136, 8896569, 21995290, 12239580, 15858239]


NameError: name 'max_idx' is not defined